In [ ]:
type Fetch = 
    | Nop 
    | AddX 
    | Value of int

let parseLine (s : string) =
    if s.StartsWith("addx") then [| AddX; Value (int (s.Substring 5)) |]
    elif s = "noop" then [|Nop|]
    else failwith "Bad opcode"

let parseCode lines =
    lines
    |> Array.collect parseLine

let apply (prevFetch : Fetch, x : int) (fetch :  Fetch) =
    match prevFetch, fetch with
    | AddX, Value v -> (fetch, v + x)
    | AddX, _ -> failwith "Bad add instruction"
    | _, _ -> (fetch, x)

let run instructions =
    instructions
    |> Array.scan apply (Nop, 1)
    |> Array.map snd

let signalStrength (output: int[]) t =
    t * output[t-1]

In [ ]:
#r "nuget: FsUnit"

open FsUnitTyped

let testProgram =
    [|
        "addx 15"
        "addx -11"
        "addx 6"
        "addx -3"
        "addx 5"
        "addx -1"
        "addx -8"
        "addx 13"
        "addx 4"
        "noop"
        "addx -1"
        "addx 5"
        "addx -1"
        "addx 5"
        "addx -1"
        "addx 5"
        "addx -1"
        "addx 5"
        "addx -1"
        "addx -35"
        "addx 1"
        "addx 24"
        "addx -19"
        "addx 1"
        "addx 16"
        "addx -11"
        "noop"
        "noop"
        "addx 21"
        "addx -15"
        "noop"
        "noop"
        "addx -3"
        "addx 9"
        "addx 1"
        "addx -3"
        "addx 8"
        "addx 1"
        "addx 5"
        "noop"
        "noop"
        "noop"
        "noop"
        "noop"
        "addx -36"
        "noop"
        "addx 1"
        "addx 7"
        "noop"
        "noop"
        "noop"
        "addx 2"
        "addx 6"
        "noop"
        "noop"
        "noop"
        "noop"
        "noop"
        "addx 1"
        "noop"
        "noop"
        "addx 7"
        "addx 1"
        "noop"
        "addx -13"
        "addx 13"
        "addx 7"
        "noop"
        "addx 1"
        "addx -33"
        "noop"
        "noop"
        "noop"
        "addx 2"
        "noop"
        "noop"
        "noop"
        "addx 8"
        "noop"
        "addx -1"
        "addx 2"
        "addx 1"
        "noop"
        "addx 17"
        "addx -9"
        "addx 1"
        "addx 1"
        "addx -3"
        "addx 11"
        "noop"
        "noop"
        "addx 1"
        "noop"
        "addx 1"
        "noop"
        "noop"
        "addx -13"
        "addx -19"
        "addx 1"
        "addx 3"
        "addx 26"
        "addx -30"
        "addx 12"
        "addx -1"
        "addx 3"
        "addx 1"
        "noop"
        "noop"
        "noop"
        "addx -9"
        "addx 18"
        "addx 1"
        "addx 2"
        "noop"
        "noop"
        "addx 9"
        "noop"
        "noop"
        "noop"
        "addx -1"
        "addx 2"
        "addx -37"
        "addx 1"
        "addx 3"
        "noop"
        "addx 15"
        "addx -21"
        "addx 22"
        "addx -6"
        "addx 1"
        "noop"
        "addx 2"
        "addx 1"
        "noop"
        "addx -10"
        "noop"
        "noop"
        "addx 20"
        "addx 1"
        "addx 2"
        "addx 2"
        "addx -6"
        "addx -11"
        "noop"
        "noop"
        "noop"
    |]

let testInstrs = parseCode testProgram
let testValues = run testInstrs
let testStrengths = [20; 60; 100; 140; 180; 220] |> List.map (signalStrength testValues)
testStrengths |> shouldEqual [420; 1140; 1800; 2940; 2880; 3960]


In [ ]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_10.txt")
let instructions = 
    File.ReadAllLines(sourcePath)
    |> parseCode

let values = run instructions

let result = 
    [20; 60; 100; 140; 180; 220] 
    |> List.sumBy (signalStrength values)


In [ ]:
printfn "Total signal strength %d" result

## Part 2

In [ ]:
let isInSprite xValues t =
    let x = xValues |> Array.item (t-1)
    abs (x-t) <= 1

let render values =
    values 
    |> Array.chunkBySize 40
    |> Array.map (fun line ->
        line 
        |> Array.mapi (fun i _ ->
            if isInSprite line (i+1) then       
                '#' 
            else 
                '.'
            )
        |> String
    )
    |> String.concat Environment.NewLine

In [ ]:
let apply2 (prevFetch : Fetch, _, counter, x : int) (fetch :  Fetch) =
    let output = 
        if abs (counter - x) <= 1  then 
            '#'
        else
            '.'
    
    let newX = 
        match prevFetch, fetch with
        | AddX, Value v -> x + v
        | AddX, _ -> failwith "Bad add instruction"
        | _, _ -> x

    let newCounter = (counter + 1) % 40
    (fetch, output, newCounter, newX)

let draw instructions =
    instructions
    |> Array.scan apply2 (Nop, '?', 0, 1)
    |> Array.map (fun (_,o,_,_)-> o)
    |> Array.skip 1

let render output =
    output
    |> Array.chunkBySize 40
    |> Array.map String
    |> String.concat Environment.NewLine

In [ ]:
testInstrs |> Array.take  1 |> draw |> render |> shouldEqual "#"
testInstrs |> Array.take  2 |> draw |> render |> shouldEqual "##"
testInstrs |> Array.take  3 |> draw |> render |> shouldEqual "##."
testInstrs |> Array.take  4 |> draw |> render |> shouldEqual "##.."
testInstrs |> Array.take  5 |> draw |> render |> shouldEqual "##..#"
testInstrs |> Array.take  6 |> draw |> render |> shouldEqual "##..##"
testInstrs |> Array.take  7 |> draw |> render |> shouldEqual "##..##."
testInstrs |> Array.take  8 |> draw |> render |> shouldEqual "##..##.."
testInstrs |> Array.take  9 |> draw |> render |> shouldEqual "##..##..#"
testInstrs |> Array.take 10 |> draw |> render |> shouldEqual "##..##..##"
testInstrs |> draw |> render

In [ ]:
instructions |> draw |> render